# Equilibrio de Nash en Juegos no cooperativos

In [ ]:
!pip3 install gurobipy

In [1]:
import numpy as np
import time
from gurobipy import *

## Equilibrio de Nash

En teoría de juegos, el equilibrio de Nash es el **concepto de solución** más utilizado para **juegos no cooperativos.**

Un **concepto de solución** es una regla formal para predecir cómo se jugará un juego. Estas predicciones se denominan "soluciones", describen las estrategias que adoptarán los jugadores y, por lo tanto, el resultado del juego.

Un **equilibrio de Nash** es un concepto de solución en el que ningún jugador puede beneficiarse al cambiar su propia estrategia (manteniendo fijas las estrategias de los demás jugadores). 

**Ejemplo:**

Si dos jugadores, Alice y Bob, eligen las estrategias A y B, (A, B) es un equilibrio de Nash si Alice no tiene ninguna otra estrategia disponible que le permita obtener un mejor resultado que A en respuesta a que Bob elija B, y Bob no tiene ninguna otra estrategia disponible que le permita obtener un mejor resultado que B en respuesta a que Alice elija A.

**Nash demostró que existe un equilibrio de Nash para cualquier juego finito.**

En este notebook, veremos una implementación de un solver para encontrar equilibrios de Nash en juegos de dos jugadores de suma cero.

## Definición

Sea $S_i$ el conjunto de todas las estrategias posibles para el jugador $i$, donde $i = 1, \dots, N$. 

Sea $s^* = (s_i^*, s_{-i}^*)$ un perfil de estrategias, un conjunto que consiste en una estrategia para cada jugador, donde $s_{-i}^*$ denota las $N-1$ estrategias de todos los jugadores excepto $i$. 

Sea $u_i(s_i, s_{-i}^*)$ la ganancia del jugador $i$ como una función de las estrategias. 

El perfil de estrategias $s^*$ es un equilibrio de Nash si

$$u_i(s_i^*, s_{-i}^*) \geq u_i(s_i, s_{-i}^*) \quad \text{para todo } s_i \in S_i.$$

Un juego puede tener más de un equilibrio de Nash. El equilibrio es único y se llama un equilibrio de Nash estricto si la desigualdad es estricta, por lo que $s_i^*$ es la única mejor respuesta:

$$u_i(s_i^*, s_{-i}^*) > u_i(s_i, s_{-i}^*) \quad \text{para todo } s_i \in S_i, s_i \neq s_i^*.$$


## Definición para dos jugadores en suma cero

Los juegos de suma cero tienen una propiedad especial que permite resolverlos: **el teorema minimax**

$$ \max_{x \in X} \min_{y \in Y} f(x, y) = \min_{y \in Y} \max_{x \in X} f(x, y) $$


A su vez, los equilibrios de Nash en juegos de suma cero con dos jugadores (2p0s) se pueden expresar un **problema de programación lineal (PPL).**

Por ello, los equilibrios pueden calcularse en tiempo polinomial.


Defina un juego
$$
G = (p_1, p_2, A_1 \times A_2, (u_1, u_2)),
$$ 

con $A_i$ el conjunto de acciones del jugador $i$ y $u_i$ la función de utilidad del jugador $i$.

Podemos entonces formular y resolver un PPL para obtener la utilidad esperada $U_1^*$  del jugador 1 y las probabilidades de acción $s_2^k$ del jugador 2 en el equilibrio de Nash.

El PPL es el siguiente:
$$
\begin{aligned}
    &\text{minimizar} && U_1^* \\
    &\text{sujeto a} && \sum_{k \in A_2} u_1(a_1^j, a_2^k) \cdot s_2^k \leq U_1^*, \quad \forall j \in A_1 \\
    &&& \sum_{k \in A_2} s_2^k = 1  \\
    &&& s_2^k \geq 0, \quad \forall k \in A_2 
\end{aligned}
$$


In [3]:
class NashEqulibriumSolver:

    def __init__(self, game_step, p1_action_num, p2_action_num, payoff_matrix):
        
        """
        game_step: step number of the game; normal game step is 1, extensive game step is more than 1
        p1_action_num: player 1's action number
        p2_action_num: player 2's action number
        payoff_matrix: the generated payoff matrix of the game
        """

        self.game_step = game_step
        self.action_choice_player1 = p1_action_num
        self.action_choice_player2 = p2_action_num
        self.action_total_num_player1 = self.action_choice_player1 ** game_step
        self.action_total_num_player2 = self.action_choice_player2 ** game_step
        self.payoff_matrix = payoff_matrix
        
        assert self.payoff_matrix.shape[0] == self.action_total_num_player1, "Payoff matrix row number does not match Player 1 pure strategy number"
        assert self.payoff_matrix.shape[1] == self.action_total_num_player2, "Payoff matrix column number does not match Player 2 pure strategy number"
    
    def solve_linear_program_player1(self, verbose=False):

        """
        Solve the NE profile strategy for player 1

        verbose: output result details or not

        Return (filtered_player1_NE, NE_utility), where filtered_player1_NE is player 1's strategy, 
                                                        and NE_utility is its NE profile utility
        """

        start_time = time.time()

        ## setup model
        LP_model = Model()
        LP_model.setParam('OutputFlag', 0)

        ## define variables
        game_value = LP_model.addVar(name='game_value', vtype=GRB.CONTINUOUS) # the NE profile utility, i.e., the value of the zero-sum two-player game
        player1_actions = LP_model.addVars(self.action_total_num_player1, vtype=GRB.CONTINUOUS) # the probabilities for all actions of player 1

        ## add constraints

        # all player 1 actions' probability should be non-negative, and their sum is 1
        LP_model.addConstr(player1_actions.sum() == 1)
        LP_model.addConstrs(player1_actions[i] >= 0 for i in range(self.action_total_num_player1))
       
        # bound player 1's utility by game_value (for each player 1's pure strategy)
        for player2_action_index in range(self.action_total_num_player2):
            lhs = 0
            for player1_action_index in range(self.action_total_num_player1):
                # if the payoff matrix is huge (e.g., converted from extensive game), we can store it with bool utilities to save space
                if type(self.payoff_matrix[player1_action_index, player2_action_index]) is np.bool_:
                    p1_utility = 1 if self.payoff_matrix[player1_action_index, player2_action_index] else -1
                # for simple normal form game, just store all payoff values
                else:
                    p1_utility = self.payoff_matrix[player1_action_index, player2_action_index]
                lhs += p1_utility * player1_actions[player1_action_index]
            LP_model.addConstr(lhs >= game_value)


        ## setup objective func and solve
        obj = game_value
        LP_model.setObjective(obj, GRB.MAXIMIZE)

        LP_model.optimize()

        ## return solved outcomes
        if LP_model.Status == GRB.Status.OPTIMAL:
            player1_NE = LP_model.getAttr('x', player1_actions)
            filtered_player1_NE = {k: v for k,v in player1_NE.items() if v > 0.}
            NE_utility = LP_model.ObjVal
            if verbose:
                print("Player 1 strategy solving time: ", time.time() - start_time)
                print("Nash Equlibrium Profile Utility for Player 1: ", NE_utility)
                print("Nash Equlibrium Player 1 Strategy: ", filtered_player1_NE)
                print("------------------------------------------")
            return (filtered_player1_NE, NE_utility)
        else:
            print(LP_model.Status, GRB.Status.OPTIMAL)
            print("The linear programming for player 1 is infeasible, please double check.")
            return None
    
    def solve_linear_program_player2(self, verbose=False):

        """
        Solve the NE profile strategy for player 2

        verbose: output result details or not

        Return (filtered_player2_NE, NE_utility), where filtered_player2_NE is player 2's strategy, 
                                                        and NE_utility is its NE profile utility
        """

        start_time = time.time()

        ## setup model
        LP_model = Model()
        LP_model.setParam('OutputFlag', 0)

        ## define variables
        game_value = LP_model.addVar(name='game_value', vtype=GRB.CONTINUOUS) # the NE profile utility, i.e., the value of the zero-sum two-player game
        player2_actions = LP_model.addVars(self.action_total_num_player2, vtype=GRB.CONTINUOUS) # the probabilities for all actions of player 2

        ## add constraints

        # all player 2 actions' probability should be non-negative, and their sum is 1
        LP_model.addConstr(player2_actions.sum() == 1)
        LP_model.addConstrs(player2_actions[i] >= 0 for i in range(self.action_total_num_player2))
       
        # bound player 2's utility by game_value (for each player 1's pure strategy)
        for player1_action_index in range(self.action_total_num_player1):
            lhs = 0
            for player2_action_index in range(self.action_total_num_player2):
                # if the payoff matrix is huge (e.g., converted from extensive game), we can store it with bool utilities to save space
                if type(self.payoff_matrix[player1_action_index, player2_action_index]) is np.bool_:
                    p1_utility = 1 if self.payoff_matrix[player1_action_index, player2_action_index] else 1
                # for simple normal form game, just store all payoff values
                else:
                    p1_utility = self.payoff_matrix[player1_action_index, player2_action_index]
                lhs += p1_utility * player2_actions[player2_action_index]
            LP_model.addConstr(lhs <= game_value)


        ## setup objective func and solve
        obj = game_value
        LP_model.setObjective(obj, GRB.MINIMIZE)

        LP_model.optimize()

        ## return solved outcomes
        if LP_model.Status == GRB.Status.OPTIMAL:
            player2_NE = LP_model.getAttr('x', player2_actions)
            filtered_player2_NE = {k: v for k,v in player2_NE.items() if v > 0.}
            NE_utility = LP_model.ObjVal
            if verbose:
                print("Player 2 strategy solving time: ", time.time() - start_time)
                print("Nash Equlibrium Profile Utility for Player 2: ", - NE_utility)
                print("Nash Equlibrium Player 2 Strategy: ", filtered_player2_NE)
                print("------------------------------------------")
            return (filtered_player2_NE, NE_utility)
        else:
            print(LP_model.Status, GRB.Status.OPTIMAL)
            print("The linear programming for player 2 is infeasible, please double check.")
            return None

In [4]:
# solve the game
def solve(game_step, p1_action_choice, p2_action_choice, payoff_matrix, verbose=False):

    # some basic setup
    p1_action_num = len(p1_action_choice)
    p2_action_num = len(p2_action_choice)

    # solving
    solver = NashEqulibriumSolver(game_step, p1_action_num, p2_action_num, payoff_matrix)
    player1_sol = solver.solve_linear_program_player1(verbose=verbose)
    player2_sol = solver.solve_linear_program_player2(verbose=verbose)

    # results
    print("-------------Player 2 strategy-------------")
    if player2_sol is not None:
        player2_strategy = player2_sol[0]
        for k, v in player2_strategy.items():
            action = p2_action_choice[k]
            print(action, v)
    else:
        print('')

    print("-------------Player 1 strategy-------------")
    if player1_sol is not None:
        player1_strategy = player1_sol[0]
        for k, v in player1_strategy.items():
            action = p1_action_choice[k]
            print(action, v)
    else:
        print('')

## Juego 1: piedra, papel o tijeras

El juego de piedra, papel o tijeras tiene la siguiente matriz de payoff:
$$
\begin{array}{c|c|c|c}
\text{p1 / p2} & \text{Rock} & \text{Paper} & \text{Scissor} \\
\hline
\text{Rock}    & (0, 0)     & (-1, +1)    & (+1, -1) \\
\text{Paper}   & (+1, -1)   & (0, 0)      & (-1, +1) \\
\text{Scissor} & (-1, +1)   & (+1, -1)    & (0, 0) \\
\end{array}
$$

In [5]:
# Definimos los objetos del juego
payoff_matrix = np.array([[0, -1, +1], 
                          [+1, 0, -1], 
                          [-1, +1, 0]])
game_step = 1
p1_actions = {0: "Rock", 1: "Paper", 2: "Scissor"}
p2_actions = {0: "Rock", 1: "Paper", 2: "Scissor"}

In [6]:
# Resolvemos el juego
solve(game_step, p1_actions, p2_actions, payoff_matrix)

Restricted license - for non-production use only - expires 2026-11-23
-------------Player 2 strategy-------------
Rock 0.3333333333333333
Paper 0.33333333333333337
Scissor 0.33333333333333337
-------------Player 1 strategy-------------
Rock 0.3333333333333333
Paper 0.3333333333333333
Scissor 0.3333333333333333


## Juego 2: matching pennies

Este juego tiene la siguiente matriz de payoff:
$$
\begin{array}{c|c|c}
\text{p1 \textbackslash p2} & \text{Cara} & \text{Cruz} \\
\hline
\text{Cara}  & (+1, -1) & (-1, +1) \\
\text{Cruz}  & (-1, +1) & (+1, -1) \\
\end{array}
$$


In [7]:
payoff_matrix = np.array([[+1, -1], [-1, +1]])
game_step = 1
p1_actions = {0: "Cara", 1: "Cruz"}
p2_actions = {0: "Cara", 1: "Cruz"}

In [8]:
# Resolvemos el juego
solve(game_step, p1_actions, p2_actions, payoff_matrix)

-------------Player 2 strategy-------------
Cara 0.5
Cruz 0.5
-------------Player 1 strategy-------------
Cara 0.5
Cruz 0.5


## Juego 3: elecciones presidenciales

Este juego tiene la siguiente matriz de payoff:
$$
\begin{array}{c|c|c}
\text{p1 \textbackslash p2} & \text{Morality} & \text{Tax-Cuts} \\
\hline
\text{Economy} & (+3, -3) & (-1, +1) \\
\text{Society} & (-2, +2) & (+1, -1) \\
\end{array}
$$

In [9]:
payoff_matrix = np.array([[+3, -1], [-2, +1]])
game_step = 1
p1_actions = {0: "Economy", 1: "Society"}
p2_actions = {0: "Morality", 1: "Tax-Cuts"}

In [10]:
# Resolvemos el juego
solve(game_step, p1_actions, p2_actions, payoff_matrix)

-------------Player 2 strategy-------------
Morality 0.2857142857142857
Tax-Cuts 0.7142857142857143
-------------Player 1 strategy-------------
Economy 0.42857142857142855
Society 0.5714285714285714


## Aplicaciones

En general, se utiliza el equilibrio de Nash para analizar el resultado de la interacción estratégica de varios tomadores de decisiones.

El concepto se ha utilizado para analizar situaciones hostiles como guerras y carreras armamentistas (*dilema del prisionero*), y también cómo el conflicto puede ser mitigado mediante interacciones repetidas (*ojo por ojo*). También se ha utilizado para estudiar hasta qué punto las personas con diferentes preferencias pueden cooperar (*batalla de los sexos*), si tomarán riesgos para lograr un resultado cooperativo (*caza de ciervos*), e incluso para penales en fútbol (*matching pennies*).



## Variantes

- **Equilibrio mixto:** No todos los jugadores siempre juegan la misma estrategia (existe una distribución de probabilidad sobre las diferentes estrategias).
- **Equilibrio no estricto**
- **Jerarquía cognitiva**

## Referencias

- https://en.wikipedia.org/wiki/Nash_equilibrium
-  https://www.cis.upenn.edu/~aaroth/courses/slides/agt24/lect07.pdf
- https://github.com/shuoyang2000/nash_equilibrium_solver